In [1]:
import control
import numpy as np
import cvxpy as cp
import scipy.optimize

In [2]:
m = 1
c = 1
k = 1

A = np.array([
    [0, 1],
    [-k/m, -c/m]
])

B = np.array([
    [0],
    [1/m]
])

C = np.array([
    [1, 0]
])

D = np.array([
    [0]
])

G = np.eye(2)

w_std = 1e-3
v_std = 1e-2

w_bound = 1e-2
v_bound = 1e-1

QN = (w_std)**2*np.eye(2)  # process noise
RN = (v_std)**2*np.eye(1)  # measurement noise
Q = np.eye(2)
R = np.eye(1)

L, _, _ = control.lqe(A, G, C, QN, RN)
K, _, _ = control.lqr(A, B, Q, R)

L = -L # flip sign for convention  A + LC
K = -K # flip sign for convention  A + BK


# check estimator and controller are stable
assert np.max(np.real(np.linalg.eig(A + L@C)[0])) < 0
assert np.max(np.real(np.linalg.eig(A + B@K)[0])) < 0

print('A', A)
print('L', L)
print('K', K)


A [[ 0.  1.]
 [-1. -1.]]
L [[-0.01484038]
 [ 0.00488988]]
K [[-0.41421356 -0.68179283]]


In [3]:
def slemma_prob(A, B, C, D, alpha0):

    # objective function
    def f(A, B, C, D, alpha):
        assert np.max(np.real(np.linalg.eig(A)[0])) < 0
        n = A.shape[0]
        m = C.shape[0]
        P = cp.Variable((n, n), 'P', PSD=True)
        mu1 = cp.Variable((1, 1), 'mu1', pos=True)
        mu2 = cp.Variable((1, 1), 'mu2', pos=True)
        constraints = [
            cp.bmat([
                [A.T@P + P@A + alpha*P, P@B],
                [B.T@P, -alpha*mu1*np.eye(n)]
            ]) << 0,
            cp.bmat([
                [C.T@C - P, C.T@D],
                [D.T@C, D.T@D - mu2*np.eye(m)]
            ]) << 0
        ]
        prob = cp.Problem(objective=cp.Minimize(mu1 + mu2), constraints=constraints)
        res = prob.solve(verbose=False)
        if prob.status == 'infeasible' or prob.status == 'infeasible_inaccurate':
            return np.inf
        elif prob.status == 'optimal' or prob.status == 'optimal_inaccurate':
            return np.sqrt(mu1.value + mu2.value)[0, 0]
        else:
            raise RuntimeError('unknown status', prob.status)

    # line search over alpha
    return scipy.optimize.minimize(
        fun=lambda x: f(A=A, B=B, C=C, D=D, alpha=x),
        x0=alpha0,
        method='Nelder-Mead')

In [4]:
# bound on: Lv + w
c_e = np.linalg.svd(L).S[0]*v_bound + w_bound  # worst case, they are aligned
c_e

0.01156252259588748

In [5]:
# C is identity here since the estimator state is the output of interest
res_est = slemma_prob(A=A+L@C, B=np.eye(2), C=np.eye(2), D=np.zeros((2, 2)), alpha0=1)
res_est


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 2.7280774298205404
             x: [ 4.930e-01]
           nit: 15
          nfev: 30
 final_simplex: (array([[ 4.930e-01],
                       [ 4.931e-01]]), array([ 2.728e+00,  2.728e+00]))

In [6]:
# estimator error bound
e_bound = c_e*res_est.fun
e_bound

0.03154345692563064

In [7]:
# bound on: BKe + w
c_bound = np.linalg.svd(B*K).S[0]*e_bound + w_bound
c_bound

0.03516397616031789

In [8]:
# want c matrix for x state, so we can find x bounds on MSD
res_ctrl = slemma_prob(A=A+B@K, B=np.eye(2), C=np.array([[1, 0]]), D=np.zeros((1, 1)), alpha0=1)
res_ctrl


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1.66728661500243
             x: [ 8.442e-01]
           nit: 13
          nfev: 26
 final_simplex: (array([[ 8.442e-01],
                       [ 8.441e-01]]), array([ 1.667e+00,  1.667e+00]))

In [9]:
x_bound = res_ctrl.fun*c_bound
x_bound

0.05862842678236256

So the MSD will be within 24 cm of reference trajectory for all time